In [ ]:
# TODO:
# WRITE A FUNCTION THAT LOADS THE FILES 
# THEN PLOT THE WINDOWS ETC. CALCULATE STATS, BASED ON WHITEBOARD
# WRITE OUT A GOOD DESCRIPTION OF WHAT ALL THE KEYS IN THE DATA TABLE ARE

In [ ]:
import numpy as np
import glob
import os
import pandas as pd



In [ ]:
#datadir = '/Users/paddyslator/OneDrive - University College London/data/PD DRUM'

cohort = 'PD'

if cohort=='PD':
    datadir = r"C:\Users\Paula\Documents\drum-hackathon\data\raw\PD DRUM"
    figdir =  r"C:\Users\Paula\Documents\drum-hackathon\data\raw\PD_figures"

    file_wildcard = "*hd-output-device*.csv"

if cohort =='HD':
    datadir = "/Users/scmps8/Library/CloudStorage/OneDrive-CardiffUniversity/data/drum/HD_DRUM"
    figdir =  "/Users/scmps8/Library/CloudStorage/OneDrive-CardiffUniversity/data/drum/HD_figures"
    
    file_wildcard = "hddrum_*/*hd-output-device*.csv"



In [ ]:
#get list of participant directories 
participant_paths = glob.glob(os.path.join(datadir,'*'))

# Initialize lists for participants with hd drum session files
participants = []

# Check each participant directory for session files
for participant_path in participant_paths:
    participant = os.path.basename(participant_path)
    
    # Define the path pattern for session files for this participant
    session_paths = glob.glob(os.path.join(datadir, participant, file_wildcard))
    
    if session_paths:  # If there are any session files
        participants.append(participant)


# Filter participant paths to include only those with session files
participant_paths = [path for path in participant_paths if os.path.basename(path) in participants]

# Check if we need to print or use valid_participants or filtered_participant_paths
print("Valid Participants:", participants)
print("Filtered Participant Paths:", participant_paths)


In [ ]:
participants

In [ ]:
#functions that get the session number, track number etc.

import os
import glob
import re
from collections import defaultdict
import pandas as pd
import io

def extract_track_number(text):
    pattern = r'track-(\d+)'
    match = re.search(pattern, text)
    if match:
        return int(match.group(1))
    else:
        return None  # or handle the case when no match is found
        
def extract_session_number(filename):
    pattern = r'session-(\d+)-track'
    match = re.search(pattern, filename)
    if match:
        return int(match.group(1))
    return None


# #function for reading the csv files while dealing with the metadata
# def read_csv_with_metadata(csv_file):
#     metadata = {}
#     data_lines = []

#     with open(csv_file, 'r') as f:
#         lines = f.readlines()

#         # Read metadata
#         for line in lines[:25]:  # assuming metadata is in the first 25 lines
#             if line.strip():  # ignore empty lines
#                 key, value = line.strip().split(',', 1)
#                 metadata[key.strip()] = value.strip()

#         # Read data table
#         start_idx = 27  # assuming data table starts from line 27
#         data_lines = lines[start_idx:]

#     # Create DataFrame for data table
#     data_table = pd.read_csv(io.StringIO(''.join(data_lines)))

#     return metadata, data_table


import pandas as pd
import io

def read_csv_with_metadata(csv_file, required_metadata_keys):
    """
    Reads the CSV file, checks if it contains the required metadata,
    and returns the metadata and data table if valid.
    """
    metadata = {}
    data_lines = []

    try:
        with open(csv_file, 'r') as f:
            lines = f.readlines()

            # Read metadata
            for line in lines[:25]:  # assuming metadata is in the first 25 lines
                if line.strip():  # ignore empty lines
                    try:
                        key, value = line.strip().split(',', 1)
                        metadata[key.strip()] = value.strip()
                    except ValueError:
                        # Skip lines that don't contain metadata in the expected format
                        continue

            # Validate metadata
            if not all(key in metadata for key in required_metadata_keys):
                print(f"Required metadata missing in file {csv_file}")
                return None, None  # Return None if required metadata is missing

            # Read data table
            start_idx = 27  # assuming data table starts from line 27
            data_lines = lines[start_idx:]

            # Create DataFrame for data table
            data_table = pd.read_csv(io.StringIO(''.join(data_lines)))

            return metadata, data_table
    except Exception as e:
        print(f"Error reading file {csv_file}: {e}")
        return None, None  # Return None if there is an error


def read_csv_with_metadata_tap(csv_file, required_metadata_keys):
    """
    Reads the CSV file, checks if it contains the required metadata,
    and returns the metadata and data table if valid.
    """
    metadata = {}
    data_lines = []

    try:
        with open(csv_file, 'r') as f:
            lines = f.readlines()

            # Read metadata
            for line in lines[:25]:  # assuming metadata is in the first 25 lines
                if line.strip():  # ignore empty lines
                    try:
                        key, value = line.strip().split(',', 1)
                        metadata[key.strip()] = value.strip()
                    except ValueError:
                        # Skip lines that don't contain metadata in the expected format
                        continue

            # Validate metadata
            if not all(key in metadata for key in required_metadata_keys):
                print(f"Required metadata missing in file {csv_file}")
                return None, None  # Return None if required metadata is missing

            # Read data table
            start_idx = 27  # assuming data table starts from line 27
            data_lines = lines[start_idx:]

            # Create DataFrame for data table
            data_table = pd.read_csv(io.StringIO(''.join(data_lines)))

            left_double_taps, right_double_taps = hit_timestamps(data_table)
            wrong_presses = 0
            wrong_presses += misses_counter(data_table)

            return metadata, data_table, left_double_taps, right_double_taps, wrong_presses
    except Exception as e:
        print(f"Error reading file {csv_file}: {e}")
        return None, None  # Return None if there is an error








# Function to filter out files starting with "~$" (temp office files)
def is_valid_file(file_path):
    return not os.path.basename(file_path).startswith('~$')

#function to clean trailing commas from metadata
def clean_metadata(metadata):
    # Define the fields that should not be converted
    non_convertible_fields = {
        'User Id', 'Date', 'Time', 'Session', 'Track', 
        'Track File', 'Metadata File'
    }
    
    cleaned_metadata = {}

    for key, value in metadata.items():
        if isinstance(value, dict):  # If the value is a nested dictionary, clean it recursively
            cleaned_value = clean_metadata(value)
        else:
            # Remove trailing commas and whitespace
            cleaned_value = value.rstrip(', ')
            
            # Convert to float if not in non_convertible_fields
            if key not in non_convertible_fields:
                try:
                    cleaned_value = float(cleaned_value)
                except ValueError:
                    print(f"Could not convert {key}: {cleaned_value} to float")
        
        # Update the cleaned_metadata dictionary
        cleaned_metadata[key] = cleaned_value

    return cleaned_metadata


import re

# def clean_metadata(metadata):
#     cleaned_metadata = {}
    
#     for key, value in metadata.items():
#         # Check if the value is a string and has extra commas
#         if isinstance(value, str):
#             # Remove extra commas and attempt to extract the primary value
#             value = re.sub(r',+', ',', value)  # Replace multiple commas with a single comma
#             parts = value.split(',')
            
#             # Take the first part as the cleaned value if it is a valid float
#             try:
#                 primary_value = float(parts[0])
#             except ValueError:
#                 print(f"Could not convert {key}: {value} to float")
#                 primary_value = None
            
#             cleaned_metadata[key] = primary_value
#         else:
#             # If the value is not a string, just add it as is
#             cleaned_metadata[key] = value
    
#     return cleaned_metadata




# def convert_metadata_values(metadata):
#     # Define the fields that should not be converted
#     non_convertible_fields = {
#         'User Id', 'Date', 'Time', 'Session', 'Track', 
#         'Track File', 'Metadata File'
#     }

#     # Iterate over the metadata dictionary
#     for key in metadata:
#         print(key)
#         for subkey in key:
#             if subkey not in non_convertible_fields:
#                 try:
#                     # Attempt to convert the value to a float
#                     metadata[key][subkey] = float(metadata[key][subkey])
#                 except ValueError:
#                     # If conversion fails, print an error message
#                     print(f"Could not convert {key}: {metadata[key][subkey]} to float")
    
#     return metadata



    

In [ ]:
# thing = os.path.join(datadir,'140223-701/hddrum_58c8dfe9c10843d8/uoc01991-hd-output-device-58c8dfe9c10843d8-session-19-track-0-2023-04-12-21-21-35.csv')

# this_session_metadata, this_session_data_table = read_csv_with_metadata(thing, required_metadata_keys)

In [ ]:
#defined the required metadata keys

# required_metadata_keys = [
#     'User Id', 'Date', 'Time', 'Session', 'Track', 'Track File', 'Metadata File',
#     'Track Duration(s)', 'Total Engagement Time(s)', 'Total Drum hits in track',
#     'Timing Window (ms)', 'Successful Drum hits', 'Success Accuracy Threshold (%)',
#     'Average Hit Accuracy (%)', 'Average Reaction Time (ms)', 'Average Left Reaction time (ms)',
#     'Average Right Reaction time (ms)', 'Best Left Reaction time (ms)', 'Best Right Reaction time (ms)',
#     'Left Accuracy (%)', 'Right Accuracy (%)', 'Left Hit Score', 'Right Hit Score'
# ]

required_metadata_keys = ['User Id', 'Date', 'Time']





In [ ]:
metadata = defaultdict(dict)
fulldata_table = defaultdict(dict)

#Initialize empty list to hold the dictionary
list_of_entries = []


for participant in participants:
    print(f"Processing participant: {participant}")
    
    # Get the file paths to the session CSV files
    session_paths = glob.glob(os.path.join(datadir, participant, file_wildcard))
    
    # Filter out files starting with "~$"
    session_paths = [path for path in session_paths if is_valid_file(path)]
    
    # Order them so that the files are in the order they were attempted
    session_paths = sorted(session_paths)
    
    # Dictionary to track attempt numbers for each session
    session_attempt_counter = defaultdict(int)
    
    for session in session_paths:
        this_session_number = extract_session_number(session)
        
        # Increment the attempt number for this session
        session_attempt_counter[this_session_number] += 1
        this_attempt_number = session_attempt_counter[this_session_number]


        try:
            # Read metadata and data table
            this_session_metadata, this_session_data_table = read_csv_with_metadata(session, required_metadata_keys)
            this_session_metadata_double, this_session_data_table_double, this_session_left_double_taps, this_session_right_double_taps, this_session_wrong_presses = read_csv_with_metadata_tap(session, required_metadata_keys)


            # Store metadata and data table
            metadata[(participant,this_session_number,this_attempt_number)] = this_session_metadata
            fulldata_table[(participant, this_session_number, this_attempt_number)] = this_session_data_table
            print(f"Processed participant: {participant}, session: {this_session_number}, attempt: {this_attempt_number}")
            list_of_entries.append([participant, this_session_number, this_attempt_number, this_session_left_double_taps, this_session_right_double_taps, this_session_wrong_presses])

        except Exception as e:
            print(f"Error processing file {session}: {e}")







In [ ]:
# Remove keys with None values directly
keys_to_remove = [k for k, v in metadata.items() if v is None]

for key in keys_to_remove:
    del metadata[key]


#clean metadata
metadata = clean_metadata(metadata)

#convert numbers in metadata to float
#metadata = convert_metadata_values(metadata)





In [ ]:
# Print metadata and data table
# print("Metadata:")
# for participant, sessions in metadata.items():
#     print(f"Participant: {participant}")
#     for session_number, metadata_info in sessions.items():
#         print(f"Session {session_number}: {metadata_info}")

print("\nFull Data Table:")
for key, data_table in fulldata_table.items():
    participant, session_number, attempt_number = key
    print(f"Participant: {participant}, Session {session_number}, Attempt {attempt_number}")
    print(data_table.head())  # assuming you want to print the first few rows of each data table
    print("\n")

In [ ]:
participants

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import math

# Number of subplots
num_plots = len(participants)

# Determine the size of the subplot grid (square-ish)
grid_size = int(math.ceil(math.sqrt(num_plots)))

# Create subplots
fig, axes = plt.subplots(grid_size, grid_size, figsize=(30, 30))

# Flatten axes array for easy iteration
axes = axes.flatten()

# Determine consistent axis limits
all_sessions = set()
all_attempts = []

# First pass: find the range of sessions and attempts
for participant in participants:
    # Filter metadata for the current participant
    participant_metadata = {key: value for key, value in metadata.items() if key[0] == participant}
    
    if participant_metadata:
        sessions = sorted(set(key[1] for key in participant_metadata.keys()))
        attempts_per_session = [0] * len(sessions)
        
        for (participant_id, session, attempt), _ in participant_metadata.items():
            session_index = sessions.index(session)
            attempts_per_session[session_index] += 1
        
        all_sessions.update(sessions)
        all_attempts.extend(attempts_per_session)

# Convert to sorted lists
all_sessions = sorted(all_sessions)
max_attempts = max(all_attempts, default=0)

# Second pass: plot data with consistent axis ticks
for i, participant in enumerate(participants):
    # Filter metadata for the current participant
    participant_metadata = {key: value for key, value in metadata.items() if key[0] == participant}
    
    if not participant_metadata:
        # If no metadata for the participant, skip this subplot
        axes[i].axis('off')
        continue
    
    # Aggregate number of attempts per session
    sessions = sorted(set(key[1] for key in participant_metadata.keys()))
    attempts_per_session = [0] * len(sessions)
    
    for (participant_id, session, attempt), _ in participant_metadata.items():
        session_index = sessions.index(session)
        attempts_per_session[session_index] += 1
    
    # Plot on the corresponding subplot
    ax = axes[i]
    ax.bar(sessions, attempts_per_session, color='skyblue')
    ax.set_title(participant[:8])  # Display only the first 5 characters
    ax.set_xlabel('Session Number')
    ax.set_ylabel('Number of Attempts')
    ax.set_xticks(all_sessions)  # Set x-axis ticks
    ax.set_xticks(np.arange(min(all_sessions), max(all_sessions) + 1, 1))  # Ensure ticks are integers
    ax.set_yticks(np.arange(0, max_attempts + 1, 1))  # Ensure y-axis ticks are integers
    ax.tick_params(axis='x', rotation=90)  # Rotate x-axis ticks by 45 degrees

    #squash down the attemps a bit
    #ax.set_ylim((0, 20))  # Ensure y-axis ticks are integers

    
# Hide any unused subplots
for j in range(num_plots, len(axes)):
    axes[j].axis('off')

# Adjust layout
plt.tight_layout()

plt.savefig(os.path.join(figdir,'attempts_per_session_by_participant.png'))

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Prepare data structure for storing average hit rates
avg_hit_rates = {}

# Compute average hit rates for each participant and session
for participant in participants:
    # Filter metadata for the current participant
    participant_metadata = {key: value for key, value in metadata.items() if key[0] == participant}
    
    # Initialize data structure for storing session hit rates
    session_hit_rates = {}

    print(participant)    
    
    for (pid, session, attempt), meta in participant_metadata.items():            
        # Get the hit accuracy
        # left_accuracy = float(meta.get('Left Accuracy (%)', 0))
        # right_accuracy = float(meta.get('Right Accuracy (%)', 0))
        # Get the hit accuracy    
        hit_accuracy = float(meta.get('Average Hit Accuracy (%)', 0))
              
        if session not in session_hit_rates:
            session_hit_rates[session] = []
            
        session_hit_rates[session].append(hit_accuracy)
    
    # Calculate average hit rate for each session
    avg_hit_rates[participant] = {session: np.mean(hit_rates) for session, hit_rates in session_hit_rates.items()}

# Number of participants
num_participants = len(avg_hit_rates)

# Determine the size of the subplot grid (square-ish)
grid_size = int(np.ceil(np.sqrt(num_participants)))

# Create subplots
fig, axes = plt.subplots(grid_size, grid_size, figsize=(30, 30))
axes = axes.flatten()  # Flatten axes array for easy iteration

# Determine the overall range for x and y axes
all_sessions = set()
all_accuracies = []

for hit_rates in avg_hit_rates.values():
    all_sessions.update(hit_rates.keys())
    all_accuracies.extend(hit_rates.values())

min_session, max_session = min(all_sessions), max(all_sessions)
min_accuracy, max_accuracy = min(all_accuracies), max(all_accuracies)

# Plotting each participant's average hit rate
for i, (participant, hit_rates) in enumerate(avg_hit_rates.items()):
    ax = axes[i]
    sessions = list(hit_rates.keys())
    accuracies = [hit_rates[session] for session in sessions]
    
    ax.bar(sessions, accuracies, color='blue', alpha=0.7)
    ax.set_title(participant[:8])  # Display only the first 5 characters of participant IDs
    ax.set_xlabel('Session Number')
    ax.set_ylabel('Average Hit Rate (%)')
    
    # Set consistent x and y axis ticks
    ax.set_xticks(np.arange(min_session, max_session + 1))
    ax.set_yticks(np.arange(0, max_accuracy + 1, step=10))
    ax.set_xlim(min_session - 0.5, max_session + 0.5)  # Adjust x limits to fit bars better
    ax.set_ylim(0, max_accuracy + 5)  # Adjust y limits to fit the bars and avoid clipping
    ax.tick_params(axis='x', rotation=90)  # Rotate x-axis ticks by 45 degrees

# Hide any unused subplots
for j in range(num_participants, len(axes)):
    axes[j].axis('off')

# Adjust layout
plt.tight_layout()

plt.savefig(os.path.join(figdir,'average_hit_rate_by_participant.png'))

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.cm as cm

# Find maximum session number
max_sessions = max([key[1] for key in metadata.keys()])

# Find overall maximum attempt number
overall_max_attempts = max([key[2] for key in metadata.keys()])

# Create subplots
fig, axs = plt.subplots(max_sessions + 1, 1, figsize=(10, 70))

# Define a colormap
cmap = cm.get_cmap('tab20', len(participants))

# Create a dictionary to store each participant's assigned color
participant_colors = {participant: cmap(i) for i, participant in enumerate(participants)}

# Store handles and labels for the legend
handles, labels = [], []

for session in range(max_sessions + 1):
    # Collect data for all participants in this session
    for participant in participants:
        attempts = []
        accuracies = []
        
        for attempt in range(1, overall_max_attempts + 1):
            key = (participant, session, attempt)
            if key in metadata:
                attempts.append(attempt)
                accuracies.append(float(metadata[key]['Average Hit Accuracy (%)']))
        
        if attempts:
            # Plot joined lines for this participant with consistent color
            participant_label = participant[:8]
            color = participant_colors[participant]
            line, = axs[session].plot(attempts, accuracies, 'o-', label=participant_label, color=color)
            
            # Collect handles and labels
            if session == 0:  # Only collect labels from the first session to avoid duplicates
                handles.append(line)
                labels.append(participant_label)

    # Set the title, labels, and x-axis limits
    axs[session].set_title(f'Session {session}')
    axs[session].set_xlabel('Attempt number')
    axs[session].set_ylabel('Hit accuracy (%)')
    #axs[session].set_xlim(0.9, overall_max_attempts)  # Set x-axis limits to ensure consistency
    axs[session].set_xlim(0.9, 20)  # Set x-axis limits to ensure consistency
    axs[session].set_xticks(range(1, 20))  # Set x-axis ticks
    axs[session].set_ylim(0, 100)  # Set y-axis limits to ensure consistency
    axs[session].tick_params(axis='x', rotation=90)  # Rotate x-axis ticks by 45 degrees

    # Add a line with the progression percentage to each plot
    progression_percentage = 70
    axs[session].plot(np.linspace(0.9, overall_max_attempts, 100), np.tile(progression_percentage, (100, 1)), 'k--')

# Create a single legend at the top
fig.legend(handles, labels, loc='upper center', ncol=5, frameon=False, title='Participants')

# Adjust layout to make space for the legend
plt.tight_layout(rect=[0, 0, 1, 0.97])  # Adjust rect to make space for the legend

plt.savefig(os.path.join(figdir, 'attempt_accuracy_tracking_by_session_cutoff.png'))


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Simulated metadata dictionary for illustration purposes
# metadata = {(participant, session, attempt): {...}, ...}

# Extract the maximum session number
max_sessions = max([key[1] for key in metadata.keys()])

# Initialize a dictionary to hold the number of attempts per session
session_attempts = {session: [] for session in range(max_sessions + 1)}

# Populate the session_attempts dictionary
for (participant, session, attempt) in metadata.keys():
    session_attempts[session].append(attempt)

# Calculate the average number of attempts for each session
average_attempts_per_session = {session: np.mean(attempts) for session, attempts in session_attempts.items()}

# Sort the session numbers
sorted_sessions = sorted(average_attempts_per_session.keys())

# Extract the corresponding average attempts
sorted_average_attempts = [average_attempts_per_session[session] for session in sorted_sessions]

# Plotting
plt.figure(figsize=(10, 6))
plt.bar(sorted_sessions, sorted_average_attempts, color='skyblue')
plt.xlabel('Session Number')
plt.ylabel('Average Number of Attempts')
#plt.title('Average Number of Attempts per Session')

# Set x-ticks to be integers
plt.xticks(ticks=np.arange(min(sorted_sessions), max(sorted_sessions) + 1))

plt.grid(axis='y')

plt.savefig(os.path.join(figdir,'attempts_per_session_overall.png'))

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from collections import Counter

# Simulated metadata dictionary for illustration purposes
# metadata = {(participant, session, attempt): {...}, ...}

# Extract participants and their sessions
participants_sessions = {}

for participant, session, attempt in metadata.keys():
    if participant not in participants_sessions:
        participants_sessions[participant] = []
    participants_sessions[participant].append(session)

# Determine the last session for each participant
last_sessions = {participant: max(sessions) for participant, sessions in participants_sessions.items()}

# Count the number of participants who failed at each session
session_fail_counts = Counter(last_sessions.values())

# Get all possible session numbers
all_sessions = range(max(session_fail_counts.keys()) + 1)

# Extract the count of failures for each session, filling in 0s for sessions with no failures
fail_counts = [session_fail_counts.get(session, 0) for session in all_sessions]

# Plotting
plt.figure(figsize=(10, 6))
plt.bar(all_sessions, fail_counts, color='skyblue')
plt.xlabel('Session Number')
plt.ylabel('Number of Participants who gave up')
#plt.title('Number of Participants who Failed at Each Session')

# Set x-ticks to be integers
plt.xticks(ticks=np.arange(min(all_sessions), max(all_sessions) + 1))

plt.grid(axis='y')

plt.savefig(os.path.join(figdir,'give_up_per_session.png'))

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Simulated metadata dictionary for illustration purposes
# metadata = {(participant, session, attempt): {...}, ...}

# Extract the maximum session number
max_sessions = max([key[1] for key in metadata.keys()])

# Initialize a dictionary to hold the hit accuracies per session
session_accuracies = {session: [] for session in range(max_sessions + 1)}

# Populate the session_accuracies dictionary
for (participant, session, attempt) in metadata.keys():
    if 'Average Hit Accuracy (%)' in metadata[(participant, session, attempt)]:
        accuracy = float(metadata[(participant, session, attempt)]['Average Hit Accuracy (%)'])
        session_accuracies[session].append(accuracy)

# Calculate the average hit accuracy for each session
average_accuracy_per_session = {session: np.mean(accuracies) for session, accuracies in session_accuracies.items()}

# Sort the session numbers
sorted_sessions = sorted(average_accuracy_per_session.keys())

# Extract the corresponding average accuracies
sorted_average_accuracies = [average_accuracy_per_session[session] for session in sorted_sessions]

# Plotting
plt.figure(figsize=(10, 6))
bars = plt.bar(sorted_sessions, sorted_average_accuracies, color='skyblue')
plt.xlabel('Session Number', fontsize=14)
plt.ylabel('Average Hit Accuracy (%)', fontsize=14)
plt.title(cohort + ' participants', fontsize=14)

# Set x-ticks to be integers
plt.xticks(ticks=np.arange(min(sorted_sessions), max(sorted_sessions) + 1))

plt.grid(axis='y')

# Add an arrow pointing vertically upwards for session 6 with rotated text and fontsize 14
session_number = 6
if session_number in sorted_sessions:
    index_of_session_6 = sorted_sessions.index(session_number)
    bar_height = sorted_average_accuracies[index_of_session_6]
    plt.annotate('off beat', 
                 xy=(session_number, bar_height),  # Arrow tip (session 6 bar)
                 xytext=(session_number, bar_height + 10),  # Text position above the bar
                 arrowprops=dict(facecolor='black', arrowstyle='->'),  # Arrow pointing upwards
                 rotation=90,  # Rotate text 90 degrees to the left
                 ha='center',  # Center horizontally
                 va='bottom',  # Align text to the bottom vertically
                 fontsize=14)  # Font size for the annotation

# Save the plot
plt.savefig(os.path.join(figdir, 'average_hit_accuracy_per_session.png'), bbox_inches='tight')
plt.savefig(os.path.join(figdir, 'average_hit_accuracy_per_session.pdf'), bbox_inches='tight')

plt.show()


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Simulated metadata dictionary for illustration purposes
# metadata = {(participant, session, attempt): {...}, ...}

# Extract the maximum session number
max_sessions = max([key[1] for key in metadata.keys()])

# Initialize a dictionary to hold the hit accuracies per session
session_accuracies = {session: [] for session in range(max_sessions + 1)}

# Populate the session_accuracies dictionary
for (participant, session, attempt) in metadata.keys():
    if 'Average Hit Accuracy (%)' in metadata[(participant, session, attempt)]:
        accuracy = float(metadata[(participant, session, attempt)]['Average Hit Accuracy (%)'])
        session_accuracies[session].append(accuracy)

# Calculate the average hit accuracy for each session
average_accuracy_per_session = {session: np.mean(accuracies) for session, accuracies in session_accuracies.items()}

# Sort the session numbers
sorted_sessions = sorted(average_accuracy_per_session.keys())

# Extract the corresponding average accuracies
sorted_average_accuracies = [average_accuracy_per_session[session] for session in sorted_sessions]

# Plotting
plt.figure(figsize=(10, 6))
bars = plt.bar(sorted_sessions, sorted_average_accuracies, color='skyblue')
plt.xlabel('Session Number', fontsize=14)
plt.ylabel('Average Hit Accuracy (%)', fontsize=14)
plt.title(cohort + ' participants', fontsize=14)

# Set x-ticks to be integers
plt.xticks(ticks=np.arange(min(sorted_sessions), max(sorted_sessions) + 1))

plt.grid(axis='y')

# Add an arrow for session 6 with text "off beat"
session_number = 6
if session_number in sorted_sessions:
    index_of_session_6 = sorted_sessions.index(session_number)
    bar_height = sorted_average_accuracies[index_of_session_6]
    plt.annotate('off beat', 
                 xy=(session_number, bar_height),  # Arrow tip (session 6 bar)
                 xytext=(session_number, bar_height + 10),  # Text position above the bar
                 arrowprops=dict(facecolor='black', arrowstyle='->'),  # Arrow pointing upwards
                 rotation=90,  # Rotate text 90 degrees to the left
                 ha='center',  # Center horizontally
                 va='bottom',  # Align text to the bottom vertically
                 fontsize=14)  # Font size for the annotation

# Add an arrow for session 8 with text "practise with both hands together"
session_number = 8
if session_number in sorted_sessions:
    index_of_session_8 = sorted_sessions.index(session_number)
    bar_height = sorted_average_accuracies[index_of_session_8]
    plt.annotate('two-handed\npractice', 
                 xy=(session_number, bar_height),  # Arrow tip (session 8 bar)
                 xytext=(session_number, bar_height + 15),  # Text position above the bar
                 arrowprops=dict(facecolor='black', arrowstyle='->'),  # Arrow pointing upwards
                 rotation=90,  # Rotate text 90 degrees to the left
                 ha='center',  # Center horizontally
                 va='bottom',  # Align text to the bottom vertically
                 fontsize=14)  # Font size for the annotation

# Add an arrow for session 10 with text "irregular pattern with off beat and reversal"
session_number = 10
if session_number in sorted_sessions:
    index_of_session_10 = sorted_sessions.index(session_number)
    bar_height = sorted_average_accuracies[index_of_session_10]
    plt.annotate('off beat\nand reversal', 
                 xy=(session_number, bar_height),  # Arrow tip (session 10 bar)
                 xytext=(session_number, bar_height + 15),  # Text position above the bar
                 arrowprops=dict(facecolor='black', arrowstyle='->'),  # Arrow pointing upwards
                 rotation=90,  # Rotate text 90 degrees to the left
                 ha='center',  # Center horizontally
                 va='bottom',  # Align text to the bottom vertically
                 fontsize=14)  # Font size for the annotation

# Add an arrow for session 14 with text "paradiddle and reversal learning"
session_number = 14
if session_number in sorted_sessions:
    index_of_session_14 = sorted_sessions.index(session_number)
    bar_height = sorted_average_accuracies[index_of_session_14]
    plt.annotate('paradiddle', 
                 xy=(session_number, bar_height),  # Arrow tip (session 14 bar)
                 xytext=(session_number, bar_height + 15),  # Text position above the bar
                 arrowprops=dict(facecolor='black', arrowstyle='->'),  # Arrow pointing upwards
                 rotation=90,  # Rotate text 90 degrees to the left
                 ha='center',  # Center horizontally
                 va='bottom',  # Align text to the bottom vertically
                 fontsize=14)  # Font size for the annotation

# Save the plot
plt.savefig(os.path.join(figdir, 'average_hit_accuracy_per_session.png'), bbox_inches='tight')
plt.savefig(os.path.join(figdir, 'average_hit_accuracy_per_session.pdf'), bbox_inches='tight')

plt.show()


In [ ]:
import matplotlib.pyplot as plt
from scipy.stats import linregress
import numpy as np

# Convert lists to numpy arrays if they're not already
sorted_average_accuracies = np.array(sorted_average_accuracies)
fail_counts = np.array(fail_counts)

# Remove positions where both arrays have zeros
mask = ~((sorted_average_accuracies == 0) & (fail_counts == 0))
sorted_average_accuracies = sorted_average_accuracies[mask]
fail_counts = fail_counts[mask]

# Scatter plot
plt.scatter(sorted_average_accuracies, fail_counts, color='k',marker='x')
plt.xlabel('Average Hit Accuracy (%)')
plt.ylabel('Number of Participants who gave up')

# Linear regression
slope, intercept, r_value, p_value, std_err = linregress(sorted_average_accuracies, fail_counts)

# Plot the regression line
plt.plot(sorted_average_accuracies, intercept + slope * sorted_average_accuracies, 'r', label=f'Linear fit, R = {r_value:.2f}, p = {p_value:.4f}')
plt.legend()

plt.title(cohort + ' participants')

# Save the plot
plt.savefig(os.path.join(figdir, 'give_up_vs_hit_accuracy.png'), bbox_inches='tight', dpi=300)
plt.savefig(os.path.join(figdir, 'give_up_vs_hit_accuracy.pdf'), bbox_inches='tight')


# Show plot
plt.show()

# Print the calculated r value and p-value
print(f'R value: {r_value}')
print(f'P value: {p_value}')






In [ ]:
p_value

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import math

# Number of subplots
num_plots = len(participants)

# Determine the size of the subplot grid (square-ish)
grid_size = int(math.ceil(math.sqrt(num_plots)))

# Create subplots
fig, axes = plt.subplots(grid_size, grid_size * 2, figsize=(15, 10))  # Two columns per participant

# Flatten axes array for easy iteration
axes = axes.flatten()

# Determine consistent axis limits for the accuracy plots
all_left_accuracies = []
all_right_accuracies = []

# First pass: collect all accuracy data
for participant in participants:
    # Filter metadata for the current participant
    participant_metadata = {key: value for key, value in metadata.items() if key[0] == participant}
    
    if participant_metadata:
        left_accuracies = [float(meta.get('Left Accuracy (%)', 0)) for (pid, session, attempt), meta in participant_metadata.items()]
        right_accuracies = [float(meta.get('Right Accuracy (%)', 0)) for (pid, session, attempt), meta in participant_metadata.items()]
        
        all_left_accuracies.extend(left_accuracies)
        all_right_accuracies.extend(right_accuracies)

max_left_accuracy = max(all_left_accuracies, default=100)
max_right_accuracy = max(all_right_accuracies, default=100)

# Second pass: plot data
for i, participant in enumerate(participants):
    # Filter metadata for the current participant
    participant_metadata = {key: value for key, value in metadata.items() if key[0] == participant}
    
    if not participant_metadata:
        # If no metadata for the participant, skip this subplot
        axes[2 * i].axis('off')
        axes[2 * i + 1].axis('off')
        continue
    
    # Extract accuracies
    left_accuracies = [float(meta.get('Left Accuracy (%)', 0)) for (pid, session, attempt), meta in participant_metadata.items()]
    right_accuracies = [float(meta.get('Right Accuracy (%)', 0)) for (pid, session, attempt), meta in participant_metadata.items()]
    
    # Plot Left Accuracy
    ax_left = axes[2 * i]
    ax_left.plot(left_accuracies, marker='o', linestyle='-', color='b')
    ax_left.set_title(f'{participant} - Left Accuracy')
    ax_left.set_xlabel('Attempt Number')
    ax_left.set_ylabel('Left Accuracy (%)')
    ax_left.set_ylim(0, max_left_accuracy)
    ax_left.grid(True)
    
    # Plot Right Accuracy
    ax_right = axes[2 * i + 1]
    ax_right.plot(right_accuracies, marker='o', linestyle='-', color='r')
    ax_right.set_title(f'{participant} - Right Accuracy')
    ax_right.set_xlabel('Attempt Number')
    ax_right.set_ylabel('Right Accuracy (%)')
    ax_right.set_ylim(0, max_right_accuracy)
    ax_right.grid(True)

# Hide any unused subplots
for j in range(num_plots * 2, len(axes)):
    axes[j].axis('off')

# Adjust layout
plt.tight_layout()

plt.savefig(os.path.join(figdir, 'left_and_right_hits.png'))


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import math

# Number of subplots
num_plots = len(participants)

# Determine the size of the subplot grid (square-ish)
grid_size = int(math.ceil(math.sqrt(num_plots)))

# Create subplots
fig, axes = plt.subplots(grid_size, grid_size * 2, figsize=(15, 10))  # Two columns per participant

# Flatten axes array for easy iteration
axes = axes.flatten()

# Determine consistent axis limits for the accuracy plots
all_left_accuracies = []
all_right_accuracies = []

# Collect all accuracy data and session boundaries
session_boundaries = {}

for participant in participants:
    # Filter metadata for the current participant
    participant_metadata = {key: value for key, value in metadata.items() if key[0] == participant}
    
    if participant_metadata:
        left_accuracies = [float(meta.get('Left Accuracy (%)', 0)) for (pid, session, attempt), meta in participant_metadata.items()]
        right_accuracies = [float(meta.get('Right Accuracy (%)', 0)) for (pid, session, attempt), meta in participant_metadata.items()]
        
        all_left_accuracies.extend(left_accuracies)
        all_right_accuracies.extend(right_accuracies)

        # Track session boundaries
        sorted_attempts = sorted((attempt for (pid, session, attempt), _ in participant_metadata.items()))
        for session in sorted(set(session for (_, session, _) in participant_metadata.keys())):
            session_boundaries[(participant, session)] = [sorted_attempts.index(min(attempt for (_, s, attempt) in participant_metadata.keys() if s == session)),
                                                          sorted_attempts.index(max(attempt for (_, s, attempt) in participant_metadata.keys() if s == session))]

max_left_accuracy = max(all_left_accuracies, default=100)
max_right_accuracy = max(all_right_accuracies, default=100)

# Plot data with shading
for i, participant in enumerate(participants):
    # Filter metadata for the current participant
    participant_metadata = {key: value for key, value in metadata.items() if key[0] == participant}
    
    if not participant_metadata:
        # If no metadata for the participant, skip this subplot
        axes[2 * i].axis('off')
        axes[2 * i + 1].axis('off')
        continue
    
    # Extract accuracies and attempts
    attempts_and_left_accuracies = [(attempt, float(meta.get('Left Accuracy (%)', 0))) for (pid, session, attempt), meta in participant_metadata.items()]
    attempts_and_right_accuracies = [(attempt, float(meta.get('Right Accuracy (%)', 0))) for (pid, session, attempt), meta in participant_metadata.items()]

    # Sort by attempts
    attempts_and_left_accuracies.sort()
    attempts_and_right_accuracies.sort()

    attempts_left, left_accuracies = zip(*attempts_and_left_accuracies)
    attempts_right, right_accuracies = zip(*attempts_and_right_accuracies)
    
    # Plot Left Accuracy
    ax_left = axes[2 * i]
    ax_left.plot(attempts_left, left_accuracies, marker='o', linestyle='-', color='b')
    
    # Shade regions based on sessions
    for (part, session), (start, end) in session_boundaries.items():
        if part == participant:
            ax_left.axvspan(attempts_left[start], attempts_left[end], color='gray', alpha=0.3, label=f'Session {session}')
    
    ax_left.set_title(f'{participant[:5]} - Left Accuracy')
    ax_left.set_xlabel('Attempt Number')
    ax_left.set_ylabel('Left Accuracy (%)')
    ax_left.set_ylim(0, max_left_accuracy)
    ax_left.grid(True)
    #ax_left.legend(loc='upper right')
    
    # Plot Right Accuracy
    ax_right = axes[2 * i + 1]
    ax_right.plot(attempts_right, right_accuracies, marker='o', linestyle='-', color='r')
    
    # Shade regions based on sessions
    for (part, session), (start, end) in session_boundaries.items():
        if part == participant:
            ax_right.axvspan(attempts_right[start], attempts_right[end], color='gray', alpha=0.3, label=f'Session {session}')
    
    ax_right.set_title(f'{participant[:5]} - Right Accuracy')
    ax_right.set_xlabel('Attempt Number')
    ax_right.set_ylabel('Right Accuracy (%)')
    ax_right.set_ylim(0, max_right_accuracy)
    ax_right.grid(True)
    #ax_right.legend(loc='upper right')

# Hide any unused subplots
for j in range(num_plots * 2, len(axes)):
    axes[j].axis('off')

# Adjust layout
plt.tight_layout()


In [ ]:
#print(fulldata_table[(participants[i], this_session_number, this_attempt_number)][])

In [ ]:
# TO DO - calculate the number of double taps etc.
import os
import numpy as np
import pandas as pd

#2 3 5 6 8 9
#i = 4
#for i in [0, 1, 4, 7, 10]:
# Convert columns to numpy arrays for easier manipulation
# expected_hits = np.array(data_table['Expected Right hit (t0)'])
expected_hits = np.array(fulldata_table[(participants[i], this_session_number, this_attempt_number)]['Expected Right hit (t0)'])

# actual_hits = np.array(data_table['Patient Right hit detected'])
actual_hits = np.array(fulldata_table[(participants[i], this_session_number, this_attempt_number)]['Patient Right hit detected'])

# window_regions = np.array(data_table['Expected Right hit window region'])
window_regions = np.array(fulldata_table[(participants[i], this_session_number, this_attempt_number)]['Expected Right hit window region'])


#subset = expected_hits[:100]

#plt.figure(figsize=(10, 4))
#plt.plot(subset, marker='o')

plt.plot(expected_hits, ".", color = "r") #[2500:2510]
plt.plot(actual_hits, ".", color = "b") #[2500:2510]
plt.plot(window_regions, ".", color = "g") #[2500:2510]


# Initialize lists to store timing accuracy, successful hits, and double taps
hit_accuracy = []
successful_hits = 0
total_expected_hits = np.sum(expected_hits)
double_taps = 0

# Function to find contiguous regions of 1's
def find_contiguous_regions(data):
    regions = []
    start = None
    for i, val in enumerate(data):
        if val == 1 and start is None:
            start = i
        elif val == 0 and start is not None:
            regions.append((start, i - 1))
            start = None
    if start is not None:
        regions.append((start, len(data) - 1))
    return regions

# Find all contiguous regions of 1's in the window region
windows = find_contiguous_regions(window_regions)

# Iterate over the windows
for start, end in windows:
    # Find the indices of expected hits within this window
    expected_indices = np.where(expected_hits[start:end+1] == 1)[0] + start
    # Find the indices of actual hits within this window
    actual_indices = np.where(actual_hits[start:end+1] == 1)[0] + start
    
    if len(expected_indices) > 0:
        # If there are actual hits, record the timing accuracy for each expected hit
        for expected_index in expected_indices:
            # Find actual hits within this window
            actual_hits_in_window = [index for index in actual_indices if start <= index <= end]
            if len(actual_hits_in_window) == 0:
                # If no actual hit in the window, append NaN
                hit_accuracy.append(np.nan)
            else:
                # Calculate the timing accuracy (absolute value) for each actual hit
                for actual_index in actual_hits_in_window:
                    timing_accuracy = actual_index - expected_index
                    hit_accuracy.append(timing_accuracy)
                    successful_hits += 1
                    # Check for double taps: if there is more than one actual hit in the window
                    if len(actual_hits_in_window) > 1:
                        double_taps += 1

# Convert the hit accuracy list to a pandas Series (or array if preferred)
hit_accuracy = pd.Series(hit_accuracy)

# Calculate the success rate
success_rate = successful_hits / total_expected_hits if total_expected_hits > 0 else 0

#deleted the max to work with the min and the max, but it shouldn't matter 

# Print the results
print("Timing accuracy:", hit_accuracy.max())
print("Number of successful hits:", successful_hits)
print("Number of double taps:", double_taps)
print("Number of double taps rate:", double_taps/successful_hits*success_rate)

print("Total expected hits:", total_expected_hits)
print("Success rate single tap:", (1-double_taps/successful_hits)*success_rate)
print(success_rate)

I have the successful_hits and the ratio between double taps and that should be the number of double taps

Then I have succesfull_rate_onetap = (1-ratiodouble)*successrate

sucessful_rate_doubletap = ratiodouble*successrate

In [ ]:
plt.hist(hit_accuracy,bins=25)

In [ ]:
print(actual_hits,type(actual_hits), actual_hits.shape)
print(expected_hits,type(expected_hits), expected_hits.shape)
print(window_regions,type(window_regions), window_regions.shape)

fulldata_table[(participant, this_session_number, this_attempt_number)]['Expected Left hit (t0)']

In [ ]:
print(fulldata_table.keys())

In [ ]:
print(fulldata_table[(participant, this_session_number, this_attempt_number)])

In [ ]:
"""CORRECT CODE"""
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

keys_to_use = fulldata_table.keys()

def find_contiguous_regions(data):
    regions = []
    start = None
    for i, val in enumerate(data):
        if val == 1 and start is None:
            start = i
        elif val == 0 and start is not None:
            regions.append((start, i - 1))
            start = None
    if start is not None:
        regions.append((start, len(data) - 1))
    return regions

all_hit_accuracies = []

for pid, session, attempt in keys_to_use:
    df = fulldata_table[(pid, session, attempt)]

    expected_hits = np.array(df['Expected Right hit (t0)'])
    actual_hits = np.array(df['Patient Right hit detected'])
    window_regions = np.array(df['Expected Right hit window region'])

    windows = find_contiguous_regions(window_regions)
    hit_accuracy = []

    for start, end in windows:
        expected_indices = np.where(expected_hits[start:end+1] == 1)[0] + start
        actual_indices = np.where(actual_hits[start:end+1] == 1)[0] + start

        for expected_index in expected_indices:
            actual_hits_in_window = [idx for idx in actual_indices if start <= idx <= end]
            if not actual_hits_in_window:
                hit_accuracy.append(np.nan)
            else:
                for actual_index in actual_hits_in_window:
                    hit_accuracy.append(actual_index - expected_index)

    all_hit_accuracies.extend(hit_accuracy)

#TODO add the double tap from joe here
"""
for start, end in windows:
    # Find the indices of expected hits within this window
    expected_indices = np.where(expected_hits[start:end+1] == 1)[0] + start
    # Find the indices of actual hits within this window
    actual_indices = np.where(actual_hits[start:end+1] == 1)[0] + start
    
    if len(expected_indices) > 0:
        # If there are actual hits, record the timing accuracy for each expected hit
        for expected_index in expected_indices:
            # Find actual hits within this window
            actual_hits_in_window = [index for index in actual_indices if start <= index <= end]
            if len(actual_hits_in_window) == 0:
                # If no actual hit in the window, append NaN
                hit_accuracy.append(np.nan)
            else:
                # Calculate the timing accuracy (absolute value) for each actual hit
                for actual_index in actual_hits_in_window:
                    timing_accuracy = actual_index - expected_index
                    hit_accuracy.append(timing_accuracy)
                    successful_hits += 1
                    # Check for double taps: if there is more than one actual hit in the window
                    if len(actual_hits_in_window) > 1:
                        double_taps += 1
"""

hit_accuracy_series = pd.Series(all_hit_accuracies).dropna()
df['Delays Right Hand'] = hit_accuracy_series
#print(df['Delays Right Hand'])
#print(df.keys)

mean_accuracy = hit_accuracy_series.mean()
std_accuracy = hit_accuracy_series.std()

plt.figure(figsize=(10, 5))
plt.hist(hit_accuracy_series, bins=30, edgecolor='black', alpha=0.7)
plt.axvline(mean_accuracy, color='red', linestyle='dashed', linewidth=1, label=f"Mean = {mean_accuracy:.2f}")
plt.axvline(mean_accuracy + std_accuracy, color='green', linestyle='dotted', linewidth=1, label=f"+1σ = {mean_accuracy + std_accuracy:.2f}")
plt.axvline(mean_accuracy - std_accuracy, color='green', linestyle='dotted', linewidth=1, label=f"-1σ = {mean_accuracy - std_accuracy:.2f}")
plt.title('Right Hand Hits')
plt.xlabel('Time')
plt.ylabel('Counts')
plt.legend()
plt.grid(True)
plt.show()

# Show accuracy statistics?
print(f"Mean: {mean_accuracy:.2f}")
print(f"sd: {std_accuracy:.2f}")


In [ ]:
"""CORRECT CODE"""
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt



keys_to_use = fulldata_table.keys()

def find_contiguous_regions(data):
    regions = []
    start = None
    for i, val in enumerate(data):
        if val == 1 and start is None:
            start = i
        elif val == 0 and start is not None:
            regions.append((start, i - 1))
            start = None
    if start is not None:
        regions.append((start, len(data) - 1))
    return regions

all_hit_accuracies = []

for pid, session, attempt in keys_to_use:
    df = fulldata_table[(pid, session, attempt)]

    expected_hits = np.array(df['Expected Left hit (t0)'])
    actual_hits = np.array(df['Patient Left hit detected'])
    window_regions = np.array(df['Expected Left hit window region'])

    windows = find_contiguous_regions(window_regions)
    hit_accuracy = []

    for start, end in windows:
        expected_indices = np.where(expected_hits[start:end+1] == 1)[0] + start
        actual_indices = np.where(actual_hits[start:end+1] == 1)[0] + start

        for expected_index in expected_indices:
            actual_hits_in_window = [idx for idx in actual_indices if start <= idx <= end]
            if not actual_hits_in_window:
                hit_accuracy.append(np.nan)
            else:
                for actual_index in actual_hits_in_window:
                    hit_accuracy.append(actual_index - expected_index)

    all_hit_accuracies.extend(hit_accuracy)

hit_accuracy_series = pd.Series(all_hit_accuracies).dropna()
df['Delays Left Hand'] = hit_accuracy_series


mean_accuracy = hit_accuracy_series.mean()
std_accuracy = hit_accuracy_series.std()

plt.figure(figsize=(10, 5))
plt.hist(hit_accuracy_series, bins=40, edgecolor='black', alpha=0.7)
plt.axvline(mean_accuracy, color='red', linestyle='dashed', linewidth=1, label=f"Mean = {mean_accuracy:.2f}")
plt.axvline(mean_accuracy + std_accuracy, color='green', linestyle='dotted', linewidth=1, label=f"+1σ = {mean_accuracy + std_accuracy:.2f}")
plt.axvline(mean_accuracy - std_accuracy, color='green', linestyle='dotted', linewidth=1, label=f"-1σ = {mean_accuracy - std_accuracy:.2f}")
plt.title('Left hand Hits')
plt.xlabel('Time')
plt.ylabel('Counts')
plt.legend()
plt.grid(True)
plt.show()

# Show accuracy statistics?
print(f"Mean: {mean_accuracy:.2f}")
print(f"sd: {std_accuracy:.2f}")

Can we check the information and see in the cases that they are really late the hit how many times they have missed?
Is there a correlation between a larger delay time and the number of times that person has missed?

To the dataframe that I have I want to add the delay time as an extra column
Then work for the same frame in a correlation between accuracy and absolute delay time

In [ ]:
mu_right = df.loc[:, 'Delays Right Hand'].mean()
sigma_right = df.loc[:, 'Delays Right Hand'].std()


#df.loc[:, 'Delays Right Hand'].mean()
s_right = np.random.normal(mu_right, sigma_right, 1000)


mu_left = df.loc[:, 'Delays Left Hand'].mean()
sigma_left = df.loc[:, 'Delays Left Hand'].std()

print(len(fulldata_table[(participant, this_session_number, this_attempt_number)]['Expected Left hit (t0)']))
X_L = np.sum(fulldata_table[(participant, this_session_number, this_attempt_number)]['Expected Left hit (t0)'] == 1)
X_R = np.sum(fulldata_table[(participant, this_session_number, this_attempt_number)]['Expected Right hit (t0)'] == 1)
#df.loc[:, 'Delays Right Hand'].mean()
s_right = np.random.normal(mu_right, sigma_right, X_R)
s_left = np.random.normal(mu_left, sigma_left, X_L)

In [ ]:
import matplotlib.pyplot as plt

count, bins, ignored = plt.hist(s_right, 30, density=True)
plt.plot(bins, 1/(sigma_right * np.sqrt(2 * np.pi)) *
               np.exp( - (bins - mu_right)**2 / (2 * sigma_right**2) ),
         linewidth=2, color='r')
plt.show()

In [ ]:
import matplotlib.pyplot as plt

count, bins, ignored = plt.hist(s_right, 30, density=True)
plt.plot(bins, 1/(sigma_left * np.sqrt(2 * np.pi)) *
               np.exp( - (bins - mu_left)**2 / (2 * sigma_left**2) ),
         linewidth=2, color='r')
plt.show()

In [ ]:
###
"""HERE"""

In [ ]:
print(actual_hits,type(actual_hits), actual_hits.shape)
print(expected_hits,type(expected_hits), expected_hits.shape)
print(window_regions,type(window_regions), window_regions.shape)

fulldata_table[(participant, this_session_number, this_attempt_number)] #['Expected Left hit (t0)']

In [ ]:
synthetic_df = pd.DataFrame({
    'Expected Left hit (t0)': fulldata_table[(participant, this_session_number, this_attempt_number)]['Expected Left hit (t0)'],
    'Expected Right hit (t0)':fulldata_table[(participant, this_session_number, this_attempt_number)]['Expected Right hit (t0)'],
    'Patient Left hit detected':fulldata_table[(participant, this_session_number, this_attempt_number)]['Patient Left hit detected'],
    'Patient Right hit detected':fulldata_table[(participant, this_session_number, this_attempt_number)]['Patient Right hit detected'],
    'Delays Right Hand': 0,
    'Delays Left Hand': 0
})

In [ ]:
import numpy as np
import pandas as pd

synthetic_df['Patient Left hit detected'] = 0
synthetic_df['Patient Right hit detected'] = 0

# Find expected hit indices
expected_left_indices = synthetic_df.index[synthetic_df['Expected Left hit (t0)'] == 1].to_numpy()
expected_right_indices = synthetic_df.index[synthetic_df['Expected Right hit (t0)'] == 1].to_numpy()

# Convert delay from ms to index offset (assuming 50 ms per row)
left_offsets = np.round(s_left).astype(int)
right_offsets = np.round(s_right).astype(int)

# Apply delays: compute target indices
left_hit_indices = expected_left_indices + left_offsets
right_hit_indices = expected_right_indices + right_offsets

# Remove out-of-bound indices
left_hit_indices = left_hit_indices[left_hit_indices < len(synthetic_df)]
right_hit_indices = right_hit_indices[right_hit_indices < len(synthetic_df)]

# Set simulated hits
synthetic_df.loc[left_hit_indices, 'Patient Left hit detected'] = 1
synthetic_df.loc[right_hit_indices, 'Patient Right hit detected'] = 1

In [ ]:
# Missed hits and hits


In [ ]:
mismatches_left = synthetic_df[
    (synthetic_df['Expected Left hit (t0)'] == 1) &
    (synthetic_df['Patient Left hit detected'] == 0)
]

# Print the first mismatch, if any
if not mismatches_left.empty:
    print("First mismatch (Left Hand):")
    print(mismatches_left.head(1))
else:
    print("No mismatches found for Left Hand.")

In [ ]:
#print(synthetic_df.iloc[1490:1500])
hits1 = synthetic_df[synthetic_df['Expected Right hit (t0)'] == 1]
print(synthetic_df.iloc[1625:1635])
#print(hits1)

In [ ]:
print(synthetic_df['Delays Right Hand'])

To the dataframe we need to add:

- Time
- Expected tap right
- Expected tap left
- Actual tap righ
- Actual tap left
- Delays ??
- For each patient separately only one track